In [6]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from tqdm import tqdm_notebook as tqdm

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
X, y = make_classification(n_samples=10000, n_features=100,
                           random_state=139, shuffle=False)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
clf = RandomForestRegressor(max_depth=2, random_state=139)
clf.fit(X_train, y_train)
print(roc_auc_score(y_test, clf.predict(X_test)))
#print(clf.feature_importances_)

0.9396798205048777


In [8]:
tr = DecisionTreeRegressor(max_depth=2)
tr.fit(X_train, y_train)

DecisionTreeRegressor(criterion='mse', max_depth=2, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [9]:
print(np.where(tr.tree_.apply(X.astype(np.float32)) == 5)[0])
print(tr.predict(X.astype(np.float32)))
print(tr.tree_.children_left)

for leaf in np.where(tr.tree_.children_left == TREE_LEAF)[0]:
    #update_terminal_region(tree, terminal_regions, leaf, X, y, residual)
    print(leaf)

[3165 3381 3779 3783 3969 4175 4524 4676 4748 5010 5014 5022 5024 5029
 5050 5063 5068 5073 5074 5089 5129 5132 5139 5140 5143 5144 5154 5184
 5196 5242 5254 5270 5290 5291 5295 5299 5303 5307 5312 5316 5320 5333
 5340 5360 5368 5372 5380 5408 5420 5426 5427 5432 5437 5442 5446 5456
 5462 5471 5473 5474 5475 5491 5495 5504 5513 5514 5530 5533 5537 5538
 5541 5554 5555 5556 5558 5572 5585 5603 5610 5611 5613 5628 5639 5654
 5664 5675 5680 5688 5689 5720 5753 5772 5773 5774 5775 5786 5787 5794
 5804 5805 5806 5810 5834 5852 5854 5876 5886 5913 5922 5931 5939 5953
 5955 5961 5973 5977 5987 5993 5994 6003 6028 6047 6057 6064 6069 6073
 6077 6088 6102 6107 6132 6138 6161 6171 6174 6181 6186 6197 6200 6205
 6220 6243 6256 6257 6258 6261 6265 6267 6275 6278 6288 6306 6307 6325
 6328 6357 6363 6368 6371 6375 6402 6406 6416 6419 6428 6460 6469 6471
 6475 6478 6479 6480 6481 6509 6514 6531 6540 6574 6578 6584 6590 6613
 6655 6668 6669 6681 6702 6730 6736 6750 6765 6780 6785 6803 6808 6822
 6827 

NameError: name 'TREE_LEAF' is not defined

In [10]:
X_train.shape

(6700, 100)

In [11]:
class DataLoader:
    def __init__(self, path):
        self.path = path
        
    def get(self):
        with open(self.path, "r") as f:
            data = np.loadtxt(f)
        y = data[:, 0] 
        X = data[:, 1:]
        return X, y

In [12]:
X_train, y_train = DataLoader("spam.train.txt").get()
X_test, y_test = DataLoader("spam.test.txt").get()

# sk_boo = GradientBoostingClassifier(n_estimators=N, random_state=139)
# sk_boo.fit(X_train, y_train)
# roc_auc_score(y_test, sk_boo.predict_proba(X_test)[:,1])


In [13]:
import numpy as np

class _Node:
    def __init__(self, node_idx):
        self.node_idx = node_idx
        self.eps = 1e-3
        self.is_leaf = False

        self.n_obj = None

        self.n_best_fea = None
        self.threshold = None

        self.left_node = None
        self.right_node = None

        self.prediction = None

    def fit(self, x, y, depth, max_depth, min_samples_split=2, min_samples_leaf=1):
        self.n_obj = x.shape[0]

        if depth == max_depth or x.shape[0] < min_samples_split:
            self.is_leaf = True
            self.prediction = np.mean(y)
            return

        x_sliced = x
        y_sliced = y
        
        # индексы отсортированных столбцов (признаков)
        idx_sliced_sorted = np.argsort(x_sliced, axis=0)
        
        # отсортированные x, y
        x_sliced_sorted = np.sort(x_sliced, axis=0)
        y_sliced_sorted = y_sliced[idx_sliced_sorted]
        
        # получаем квадраты целей и аккумулируем их в последней строке
        y_sq_cumsum = np.cumsum(y_sliced_sorted ** 2, axis=0)
        # то же на развернутой матрице
        y_sq_cumsum_reversed = np.cumsum(y_sliced_sorted[::-1] ** 2, axis=0)[::-1]
        
        # аккумуляция первых степеней
        y_cumsum = np.cumsum(y_sliced_sorted, axis=0)
        # аккумуляция первых степеней в развернутой матрице
        y_cumsum_reversed = np.cumsum(y_sliced_sorted[::-1], axis=0)[::-1]

        # считаем эмпирическую дисперсию (умноженную на n)
        cnt_array = np.array(range(1, y_cumsum.shape[0] + 1)).reshape(-1, 1)
        mse_left = y_sq_cumsum - y_cumsum ** 2 / cnt_array
        mse_right = y_sq_cumsum_reversed - y_cumsum_reversed ** 2 / cnt_array[::-1]
        
        # скалдываем левый и правый вариант mse 
        mse = mse_left
        mse[:-1] += mse_right[1:]

        # исключаем из рассмотрения незначительные признаки (заполняя их nan)
        garbage_fea = np.where(np.abs(np.max(x, axis=0) - np.min(x, axis=0)) < self.eps)[0]
        if garbage_fea.shape[0] > 0:
            mse[:, garbage_fea] = np.nan
        # исключаем последнюю строку
        mse[-1] = np.nan
        # исключаем случаи разделения по признакам с равным значениями mse
        mse[x_sliced_sorted == np.roll(x_sliced_sorted, shift=-1, axis=0)] = np.nan
        
        # получаем двумерные индексы в матрице mse 
        try:
            n_best_obj, self.n_best_fea = np.unravel_index(np.nanargmin(mse), mse.shape)
        except ValueError:
            self.is_leaf = True
            self.prediction = np.mean(y)
            return
        
        # фиксируем порог
        self.threshold = x_sliced_sorted[n_best_obj, self.n_best_fea]

        # получаем индексы левой и правой подматрице
        left_obj = np.where(x[:, self.n_best_fea] <= self.threshold)[0]
        right_obj = np.where(x[:, self.n_best_fea] > self.threshold)[0]
        if left_obj.shape[0] < min_samples_leaf or right_obj.shape[0] < min_samples_leaf:
            self.is_leaf = True
            self.prediction = np.mean(y)
            return
        
        self.left_node = _Node(2 * self.node_idx + 1)
        self.right_node = _Node(2 * self.node_idx + 2)
        self.left_node.fit(x[left_obj], y[left_obj], depth + 1, max_depth, min_samples_split)
        self.right_node.fit(x[right_obj], y[right_obj], depth + 1, max_depth, min_samples_split)

    def predict(self, x):
        if self.is_leaf:
            return self.prediction

        prediction = np.empty([x.shape[0]])
        left_obj = np.where(x[:, self.n_best_fea] <= self.threshold)
        right_obj = np.where(x[:, self.n_best_fea] > self.threshold)

        prediction[left_obj] = self.left_node.predict(x[left_obj])
        prediction[right_obj] = self.right_node.predict(x[right_obj])

        return prediction
    
    def apply(self, x):
        if self.is_leaf:
            return self.node_idx
        
        idx = np.empty([x.shape[0]], dtype=np.int32)
        left_obj = np.where(x[:, self.n_best_fea] <= self.threshold)
        right_obj = np.where(x[:, self.n_best_fea] > self.threshold)

        idx[left_obj] = self.left_node.apply(x[left_obj])
        idx[right_obj] = self.right_node.apply(x[right_obj])

        return idx

    def traverse(self, nodes_list):
        nodes_list.append(self)
        if self.is_leaf:
            return
        self.left_node.traverse(nodes_list)
        self.right_node.traverse(nodes_list)
            
        
class RegressionDecisionTree:
    def __init__(self, max_depth=1, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split

        self.top_node = _Node(node_idx=0)
        self.nodes = []

    def fit(self, x, y):
        self.top_node.fit(x, y, 0, self.max_depth, self.min_samples_split)
        self.traverse()

    def predict(self, x):
        return self.top_node.predict(x)

    def traverse(self):
        return self.top_node.traverse(self.nodes)
        
    def __getitem__(self, key):
        for item in self.nodes:
            if item.node_idx == key:
                return item
    

In [6]:
clf = RegressionDecisionTree()
sk_clf = DecisionTreeRegressor()
clf.fit(X_train.astype(np.float64),y_train.astype(np.float64))
print(roc_auc_score(y_test, clf.predict(X_test)))
sk_clf.fit(X_train.astype(np.float64),y_train.astype(np.float64))
print(roc_auc_score(y_test, sk_clf.predict(X_test)))
#print(clf.tree_.apply(X_test))
#print(clf.leafs)
print(clf.nodes)

0.8038803712322645
0.7920565192582394
[<__main__._Node instance at 0x7ff699a91368>, <__main__._Node instance at 0x7ff663b3f0e0>, <__main__._Node instance at 0x7ff69a701b00>]


In [8]:
class Bagging(object):
    
    def __init__(self, max_depth = 3, n_estimators = 30, subsample_size = 5000, max_features=100):
            
        self.max_depth = max_depth
        self.n_estimators = n_estimators
        self.subsample_size = subsample_size
        self.max_features = max_features
#         self.estimators_ = [DecisionTreeRegressor(max_depth=max_depth, random_state=139)
#                            for _ in range(self.n_estimators)]
        self.estimators_ = [RegressionDecisionTree(max_depth=max_depth)
                               for _ in range(self.n_estimators)]
        self.rsm_indices = []
    
    def get_bootstrap_samples(self, X, y):
        indices = np.random.choice(X.shape[0], self.subsample_size, replace=True)
        rsm_indices = np.random.choice(X.shape[1], self.max_features, replace=False)
        #rsm_indices = np.arange(int(np.sqrt(X.shape[1])))
        #self.rsm_indices.append(rsm_indices)
        samples_X = X[indices,:]#[:,rsm_indices]
        samples_y = y_train[indices]
        return samples_X, samples_y

#         n_samples = X.shape[0]

#         indices = np.random.randint(0, n_samples, n_samples)
#         samples_X = X[indices,:]
#         samples_y = y_train[indices]
        
#         return samples_X, samples_y
    
    def fit(self, data_train, y_train):        
        for i in range(self.n_estimators):
            samples_train, samples_y = self.get_bootstrap_samples(data_train, y_train)
            self.estimators_[i].built_tree = self.estimators_[i].fit(samples_train.astype(np.float64), samples_y.astype(np.float64))
                
    def predict(self, X_test):      
        pred = []
        for i in range(self.n_estimators):
            #pred.append(self.estimators_[i].predict(X_test[:, self.rsm_indices[i]].astype(np.float64)))
            pred.append(self.estimators_[i].predict(X_test.astype(np.float64)))
        return np.mean(pred, axis=0)

    

In [14]:
def sigmoid(h):
    return 1./(1. + np.exp(-h))

class Boosting(object):
    def __init__(self):
        self.boo = []
        
    def append(self, bag):
        self.boo.append(bag)
        
    def predict(self, X):
        prediction = np.zeros(X.shape[0])
        for bag in self.boo:
            prediction += bag.predict(X)
        return prediction
    
    def _dLdh(self, y_true, y_pred):
        return sigmoid(y_pred) - y_true
    
    def next_values(self, y_true, y_pred):
        return -self._dLdh(y_true, y_pred)
    

In [18]:

def update_terminal_region(terminal_regions, leaf, X, y, residual):
        """Make a single Newton-Raphson step.

        our node estimate is given by:

            sum((y - prob)) / sum(prob * (1 - prob))

        we take advantage that: y - prob = residual
        """
        terminal_region = np.where(terminal_regions.astype(np.int32) == leaf.node_idx)[0]
        residual = residual.take(terminal_region, axis=0)
        y = y.take(terminal_region, axis=0)

        numerator = np.sum(residual)
        denominator = np.sum((y - residual) * (1 - y + residual))

        # prevents overflow and division by zero
        if abs(denominator) < 1e-150:
            leaf.prediction = 0.0
        else:
            leaf.prediction = numerator / denominator


def optimize_leafs(forest, X, y, residual):
    tree = forest
    #for tree in bag.estimators_:
    terminal_regions = tree.top_node.apply(X.astype(np.float32))
    for node in tree.nodes:
        if node.is_leaf:
            update_terminal_region(terminal_regions, node, X, y, residual)

In [ ]:
# sklearn interface
# from sklearn.tree._tree import TREE_LEAF

# def update_terminal_region(tree, terminal_regions, leaf, X, y, residual):
#         """Make a single Newton-Raphson step.

#         our node estimate is given by:

#             sum((y - prob)) / sum(prob * (1 - prob))

#         we take advantage that: y - prob = residual
#         """
#         terminal_region = np.where(terminal_regions == leaf)[0]
#         residual = residual.take(terminal_region, axis=0)
#         y = y.take(terminal_region, axis=0)

#         numerator = np.sum(residual)
#         denominator = np.sum((y - residual) * (1 - y + residual))

#         # prevents overflow and division by zero
#         if abs(denominator) < 1e-150:
#             tree.value[leaf, 0, 0] = 0.0
#         else:
#             tree.value[leaf, 0, 0] = numerator / denominator


# def optimize_leafs(forest, X, y, residual):
#     for tree in bag.estimators_:
#         tree = tree.tree_
#         terminal_regions = tree.apply(X.astype(np.float32))
#         for leaf in np.where(tree.children_left == TREE_LEAF)[0]:
#             update_terminal_region(tree, terminal_regions, leaf, X, y, residual)


In [16]:
def LogLoss(y_true, y_pred):
    return np.sum(-y_true*np.log(sigmoid(y_pred)) - (1 - y_true)*np.log(1 - sigmoid(y_pred)))

In [19]:
N = 20
boo = Boosting()
labels_to_predict = y_train
h_i = np.zeros(X_train.shape[0])
eta = 0.1

#bag = RandomForestRegressor(max_depth=3, random_state=139)# обучаем лесок на текущих метках
#bag = Bagging(subsample_size = 7093, max_features=102)
bag = RegressionDecisionTree(max_depth=3)
bag.fit(X_train, labels_to_predict)
boo.append(bag)
for idx in tqdm(range(N)):  
    h_i = h_i + bag.predict(X_train)
    #print(np.sum(boo._dLdh(y_train, h_i)))
    labels_to_predict = boo.next_values(y_train, h_i)# делаем шаг бустинга
    
    #bag = RandomForestRegressor(max_depth=3, random_state=139)# обучаем лесок на текущих метках
    #bag = DecisionTreeRegressor(max_depth=3, random_state=139)
    #bag = Bagging(n_estimators=50, subsample_size = 1000, max_features=60)
    bag = RegressionDecisionTree(max_depth=3)
    bag.fit(X_train, labels_to_predict)
    boo.append(bag)
    
    optimize_leafs(bag, X_train, y_train, labels_to_predict)

    
    
print(roc_auc_score(y_train, boo.predict(X_train)),
      roc_auc_score(y_test, boo.predict(X_test)))

sk_boo = GradientBoostingClassifier(n_estimators=N, random_state=139)
sk_boo.fit(X_train, y_train)
roc_auc_score(y_test, sk_boo.predict_proba(X_test)[:,1])


(0.9972623396222717, 0.8494186873829722)


0.9692422651821415

In [ ]:
sk_boo.predict_proba(X_test)[:, 1]

In [ ]:
y_test

In [ ]:
print(boo.predict(X_test))

In [14]:
print("Test")
print(LogLoss(y_test, boo.predict(X_test)))
print(LogLoss(y_test, sk_boo.decision_function(X_test)))
print(LogLoss(y_test, boo.predict(X_test)) / LogLoss(y_test, sk_boo.decision_function(X_test)))

print("Train")
print(LogLoss(y_train, boo.predict(X_train)))
print(LogLoss(y_train, sk_boo.decision_function(X_train)))
print(LogLoss(y_train, boo.predict(X_train)) / LogLoss(y_train, sk_boo.decision_function(X_train)))


Test
3297.913642047876
3034.095319676036
1.0869512307873073
Train
1015.4797183306739
1625.8383504320245
0.6245883657873099


In [ ]:
np.random.choice(X.shape[1], 100, replace=True)

In [ ]:
bag = Bagging()
bag.fit(X_train, y_train)
roc_auc_score(y_test, bag.predict(X_test))

In [ ]:
boo.boo

## Передерываю

In [4]:
import numpy as np

class _Node:
    def __init__(self, node_idx):
        self.node_idx = node_idx
        self.eps = 1e-3
        self.is_leaf = False

        self.n_obj = None

        self.n_best_fea = None
        self.threshold = None

        self.left_node = None
        self.right_node = None

        self.prediction = None

    def fit(self, x, y, depth, max_depth, min_samples_split=2, min_samples_leaf=1):
        self.n_obj = x.shape[0]

        if depth == max_depth or x.shape[0] < min_samples_split:
            self.is_leaf = True
            self.prediction = np.mean(y)
            return

        x_sliced = x
        y_sliced = y
        
        # индексы отсортированных столбцов (признаков)
        idx_sliced_sorted = np.argsort(x_sliced, axis=0)
        
        # отсортированные x, y
        x_sliced_sorted = np.sort(x_sliced, axis=0)
        y_sliced_sorted = y_sliced[idx_sliced_sorted]
        
        # получаем квадраты целей и аккумулируем их к последней строке
        y_sq_cumsum = np.cumsum(y_sliced_sorted ** 2, axis=0)
        # то же на развернутой матрице
        y_sq_cumsum_reversed = np.cumsum(y_sliced_sorted[::-1] ** 2, axis=0)[::-1]
        
        # аккумуляция первых степеней
        y_cumsum = np.cumsum(y_sliced_sorted, axis=0)
        # аккумуляция первых степеней в развернутой матрице
        y_cumsum_reversed = np.cumsum(y_sliced_sorted[::-1], axis=0)[::-1]

        # считаем эмпирическую дисперсию (умноженную на n)
        cnt_array = np.array(range(1, y_cumsum.shape[0] + 1)).reshape(-1, 1)
        mse_left = y_sq_cumsum - y_cumsum ** 2 / cnt_array
        mse_right = y_sq_cumsum_reversed - y_cumsum_reversed ** 2 / cnt_array[::-1]
        
        # скалдываем левый и правый вариант mse 
        mse = mse_left
        mse[:-1] += mse_right[1:]

        # исключаем из рассмотрения незначительные признаки (заполняя их nan)
        garbage_fea = np.where(np.abs(np.max(x, axis=0) - np.min(x, axis=0)) < self.eps)[0]
        if garbage_fea.shape[0] > 0:
            mse[:, garbage_fea] = np.nan
        # исключаем последнюю строку
        mse[-1] = np.nan
        # исключаем случаи разделения по признакам с равным значениями mse
        mse[x_sliced_sorted == np.roll(x_sliced_sorted, shift=-1, axis=0)] = np.nan
        
        # получаем двумерные индексы в матрице mse 
        try:
            n_best_obj, self.n_best_fea = np.unravel_index(np.nanargmin(mse), mse.shape)
        except ValueError:
            self.is_leaf = True
            self.prediction = np.mean(y)
            return
        
        # фиксируем порог
        self.threshold = (x_sliced_sorted[n_best_obj, self.n_best_fea] + 
                            x_sliced_sorted[n_best_obj + 1, self.n_best_fea]) / 2.0

        # получаем индексы левой и правой подматрице
        left_obj = np.where(x[:, self.n_best_fea] <= self.threshold)[0]
        right_obj = np.where(x[:, self.n_best_fea] > self.threshold)[0]
        if left_obj.shape[0] < min_samples_leaf or right_obj.shape[0] < min_samples_leaf:
            self.is_leaf = True
            self.prediction = np.mean(y)
            return
        
        self.left_node = _Node(2 * self.node_idx + 1)
        self.right_node = _Node(2 * self.node_idx + 2)
        self.left_node.fit(x[left_obj], y[left_obj], depth + 1, max_depth, min_samples_split)
        self.right_node.fit(x[right_obj], y[right_obj], depth + 1, max_depth, min_samples_split)

    def predict(self, x):
        if self.is_leaf:
            return self.prediction

        prediction = np.empty([x.shape[0]])
        left_obj = np.where(x[:, self.n_best_fea] <= self.threshold)
        right_obj = np.where(x[:, self.n_best_fea] > self.threshold)

        prediction[left_obj] = self.left_node.predict(x[left_obj])
        prediction[right_obj] = self.right_node.predict(x[right_obj])

        return prediction
    
    def apply(self, x):
        if self.is_leaf:
            return self.node_idx
        
        idx = np.empty([x.shape[0]], dtype=np.int32)
        left_obj = np.where(x[:, self.n_best_fea] <= self.threshold)
        right_obj = np.where(x[:, self.n_best_fea] > self.threshold)

        idx[left_obj] = self.left_node.apply(x[left_obj])
        idx[right_obj] = self.right_node.apply(x[right_obj])

        return idx

    def traverse(self, nodes_list):
        nodes_list.append(self)
        if self.is_leaf:
            return
        self.left_node.traverse(nodes_list)
        self.right_node.traverse(nodes_list)
            
        
class RegressionDecisionTree:
    def __init__(self, max_depth=1, min_samples_split=2):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split

        self.top_node = _Node(node_idx=0)
        self.nodes = []

    def fit(self, x, y):
        self.top_node.fit(x, y, 0, self.max_depth, self.min_samples_split)
        self.traverse()

    def predict(self, x):
        return self.top_node.predict(x)

    def traverse(self):
        return self.top_node.traverse(self.nodes)
        
    def __getitem__(self, key):
        for item in self.nodes:
            if item.node_idx == key:
                return item
    